In [1]:
import os
os.chdir('/home/jagust/dino/surf/')
from surf import *
%matplotlib inline

In [3]:
young_neg_mat = np.load('/home/jagust/dino/surf/mats/young_neg_med_mat.npy')
young_pos_mat = np.load('/home/jagust/dino/surf/mats/young_pos_med_mat.npy')
young_mat = np.load('/home/jagust/dino/surf/mats/young_med_mat.npy')
old_neg_mat = np.load('/home/jagust/dino/surf/mats/ignore_pib_pos/neg_med_mat.npy')
old_pos_mat = np.load('/home/jagust/dino/surf/mats/ignore_pib_pos/pos_med_mat.npy')
matches = np.load('/home/jagust/dino/surf/matches.npy')

In [21]:
old_neg_g, old_pos_g = nx.Graph(old_neg_mat), nx.Graph(old_pos_mat)
add_distance_attr(old_neg_g)
add_distance_attr(old_pos_g)
old_neg_mst, old_pos_mst = nx.minimum_spanning_tree(old_neg_g, weight='distance'), nx.minimum_spanning_tree(old_pos_g, weight='distance')
old_neg_edges, old_pos_edges = set(old_neg_mst.edges()), set(old_pos_mst.edges())
old_diff = neg_edges.difference(pos_edges)

In [22]:
young_neg_g, young_pos_g = nx.Graph(young_neg_mat), nx.Graph(young_pos_mat)
add_distance_attr(young_neg_g)
add_distance_attr(young_pos_g)
young_neg_mst, young_pos_mst = nx.minimum_spanning_tree(young_neg_g, weight='distance'), nx.minimum_spanning_tree(young_pos_g, weight='distance')
young_neg_edges, young_pos_edges = set(young_neg_mst.edges()), set(young_pos_mst.edges())
young_diff = neg_edges.difference(pos_edges)

In [35]:
pos_pib_dat = np.load('/home/jagust/dino/surf/mats/ignore_pib_pos/pos_pib_dat.npy')
neg_pib_dat = np.load('/home/jagust/dino/surf/mats/ignore_pib_pos/neg_pib_dat.npy')
pib_dat = np.vstack((pos_pib_dat, neg_pib_dat))

pib_vals = pd.DataFrame(pib_dat, dtype = float)
corr_mat = np.array(pib_vals.corr())
np.fill_diagonal(corr_mat, 0.0)
pib_g = nx.Graph(corr_mat)

In [39]:
add_distance_attr(pib_g)
pib_mst = nx.minimum_spanning_tree(pib_g, weight='distance')
pib_edges = set(pib_mst.edges())

In [23]:
len(young_diff), len(old_diff)

(58, 58)

In [24]:
diff_diff = young_diff.difference(old_diff)
diff_diff

set()

In [25]:
neg_diff = old_neg_edges.difference(young_neg_edges)
pos_diff = old_pos_edges.difference(young_pos_edges)

In [27]:
len(neg_diff), len(pos_diff)

(57, 50)

In [30]:
diff_diff_diff = neg_diff.difference(pos_diff)
len(diff_diff_diff)

51

In [41]:
len(pib_edges.difference(old_neg_edges))

70

In [42]:
len(pib_edges.difference(old_pos_edges))

71

In [2]:
def median(l):
    return np.median(np.array(l))

def make_median_mat(paths):
    group_mat = np.zeros((80, 80))
    mats = [np.load(path) for path in paths]
    for r in range(80):
        for c in range(80)[r:80]:
            vals = []
            for mat in mats:
                vals.append(mat[r][c])
            vals = sorted(vals)
            group_mat[r][c] = median(vals)
            group_mat[c][r] = median(vals)
    return group_mat
def load_graph(path):
    if type(path) == str or type(path) == unicode:
        g = nx.Graph(np.load(path))
        add_distance_attr(g)
    else:
        g = nx.Graph(path)
        add_distance_attr(g)
    return g